In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
!ls -l ../data/

total 580596
-rw------- 1 v.bugaevskii v.bugaevskii    904039 Apr 17 11:12 appl.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6577695 Apr 17 11:12 aum.csv
-rw------- 1 v.bugaevskii v.bugaevskii  91636746 Apr 17 11:12 balance.csv
-rw------- 1 v.bugaevskii v.bugaevskii   1110765 Apr 17 11:12 client.csv
-rw------- 1 v.bugaevskii v.bugaevskii   6686165 Apr 17 11:12 com.csv
-rw------- 1 v.bugaevskii v.bugaevskii   7499804 Apr 17 11:12 deals.csv
-rw------- 1 v.bugaevskii v.bugaevskii     18979 Apr 17 11:12 dict_mcc.csv
-rw------- 1 v.bugaevskii v.bugaevskii   2322389 Apr 17 11:12 funnel.csv
-rw------- 1 v.bugaevskii v.bugaevskii  10553763 Apr 17 11:12 payments.csv
-rw------- 1 v.bugaevskii v.bugaevskii 467193115 Apr 17 11:12 trxn.csv


In [3]:
from pathlib import Path

In [4]:
path_df = Path('../data').rglob('*.csv')
path_df = sorted(path_df)

for path in tqdm(path_df):
    df_name = 'df_' + path.name.rsplit('.', 1)[0]
    vars()[df_name] = pd.read_csv(path)
    print(f'{path} saved to {df_name}; {df_name}.shape = {vars()[df_name].shape}')

  0%|          | 0/10 [00:00<?, ?it/s]

../data/appl.csv saved to df_appl; df_appl.shape = (12030, 6)
../data/aum.csv saved to df_aum; df_aum.shape = (117392, 4)
../data/balance.csv saved to df_balance; df_balance.shape = (1194684, 9)
../data/client.csv saved to df_client; df_client.shape = (21498, 8)
../data/com.csv saved to df_com; df_com.shape = (113055, 10)
../data/deals.csv saved to df_deals; df_deals.shape = (109016, 8)
../data/dict_mcc.csv saved to df_dict_mcc; df_dict_mcc.shape = (915, 3)
../data/funnel.csv saved to df_funnel; df_funnel.shape = (21498, 16)
../data/payments.csv saved to df_payments; df_payments.shape = (188068, 4)


/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


../data/trxn.csv saved to df_trxn; df_trxn.shape = (3035705, 11)


In [5]:
df_dict_mcc['mcc_category'] = df_dict_mcc['brs_mcc_group'].astype('category').cat.codes
df_dict_mcc.head()

,mcc_cd,brs_mcc_group,brs_mcc_subgroup,mcc_category
0,3658,Travel,Hotels,20
1,3659,Travel,Hotels,20
2,3660,Travel,Hotels,20
3,3661,Travel,Hotels,20
4,3662,Travel,Hotels,20


In [6]:
df_trxn['tran_time'] = pd.to_datetime(df_trxn['tran_time'])
df_trxn['tran_time_day_offset'] = (df_trxn['tran_time'] - df_trxn['tran_time'].min()).dt.days
df_trxn['mcc_cd'] = df_trxn['mcc_cd'].fillna(-1).astype(int)

In [7]:
df = pd.merge(df_trxn, df_dict_mcc[['mcc_cd', 'mcc_category']], how='left', on='mcc_cd')
df['mcc_category'] = df['mcc_category'].fillna(-1).astype(int)
df['txn_comment_1_cat'] = df['txn_comment_1'].astype('category').cat.codes
df['txn_comment_2_cat'] = df['txn_comment_2'].astype('category').cat.codes
df.head()

,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2,tran_time_day_offset,mcc_category,txn_comment_1_cat,txn_comment_2_cat
0,-1893104556496814867,-6909844117230568380,2019-07-01 00:00:00,6071.0,6011,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN,303,2,2,-1
1,-1893104556496814867,-6909844117230568380,2019-07-29 00:00:00,6071.0,6011,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN,331,2,2,-1
2,-1893104556496814867,-6909844117230568380,2019-08-31 00:00:00,7285.0,6011,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN,364,2,2,-1
3,6886062013213911831,6527755410945490683,2019-08-14 10:38:33,2428.0,6011,-774579209084196290,RUS,RADUZHNYI,TTS SITI,Cash withdrawal through an ATM,NaN,347,2,2,-1
4,6886062013213911831,6527755410945490683,2019-07-17 12:27:58,6071.0,6011,-7735007003326589238,RUS,RADUZHNYI,ODTS AGANGRAD,Cash withdrawal through an ATM,NaN,319,2,2,-1


In [8]:
df['txn_comment_1'].value_counts()

Payment for goods and services     2546655
Cash withdrawal through an ATM      243562
Cashless transfer                   146586
Payment by card (bank transfer)      52325
Cash deposit by card                 36342
Return of goods / services            8979
Cash withdrawal                       1256
Name: txn_comment_1, dtype: int64

In [9]:
df['txn_comment_2'].value_counts()

<manual mode>             157751
Opening Online            138906
Purchase payment (web)     93744
VISA MONEY TRANSFER         6511
MASTERCARD MONEYSEND           7
Name: txn_comment_2, dtype: int64

In [10]:
df[df['txn_comment_2'] == '<manual mode>']

,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2,tran_time_day_offset,mcc_category,txn_comment_1_cat,txn_comment_2_cat
595681,-4976026508164850392,-3405418671908513727,2019-08-26 08:42:38,3642.0,6012,3967370607447365704,RUS,MOSKVA,QPAYMENT SYSTEM,Payment for goods and services,<manual mode>,359,8,5,0
595682,-4976026508164850392,-3405418671908513727,2019-08-25 06:28:16,3642.0,6012,4800355837163983220,RUS,YAROSLAVL,FPNS..,Payment for goods and services,<manual mode>,358,8,5,0
595683,-4976026508164850392,-3405418671908513727,2019-08-25 22:31:42,3642.0,6012,3967370607447365704,RUS,MOSKVA,QPAYMENT SYSTEM,Payment for goods and services,<manual mode>,358,8,5,0
595684,-4976026508164850392,-3405418671908513727,2019-07-09 21:01:37,2428.0,6012,3967370607447365704,RUS,PETROZAVODSK,OZPL.,Payment for goods and services,<manual mode>,311,8,5,0
595685,-4976026508164850392,-3405418671908513727,2019-08-26 08:48:17,3642.0,6012,3967370607447365704,RUS,MOSKVA,QPAYMENT SYSTEM,Payment for goods and services,<manual mode>,359,8,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033799,-174115151336149439,2330452560717342967,2018-09-09 00:00:00,60706.0,6012,3967370607447365704,RUS,VISA DIRECT,TINKOFF BANK CARDCARD,Cashless transfer,<manual mode>,8,8,3,0
3033800,-174115151336149439,2330452560717342967,2018-10-26 00:00:00,182118.0,6012,3967370607447365704,RUS,VISA DIRECT,TINKOFF BANK CARDCARD,Cashless transfer,<manual mode>,55,8,3,0
3033801,-174115151336149439,2330452560717342967,2018-12-30 00:00:00,60706.0,6012,3967370607447365704,RUS,VISA DIRECT,TINKOFF BANK CARDCARD,Cashless transfer,<manual mode>,120,8,3,0
3033802,-174115151336149439,2330452560717342967,2018-11-07 00:00:00,60706.0,6012,3967370607447365704,RUS,VISA DIRECT,TINKOFF BANK CARDCARD,Cashless transfer,<manual mode>,67,8,3,0


In [11]:
MCC_CODES_RARE = {
    'mcc_cd_-1',
    'mcc_cd_1520',
    'mcc_cd_1711',
    'mcc_cd_1731',
    'mcc_cd_1740',
    'mcc_cd_1750',
    'mcc_cd_1761',
    'mcc_cd_1799',
    'mcc_cd_2741',
    'mcc_cd_2791',
    'mcc_cd_2842',
    'mcc_cd_3007',
    'mcc_cd_3008',
    'mcc_cd_3011',
    'mcc_cd_3015',
    'mcc_cd_3026',
    'mcc_cd_3034',
    'mcc_cd_3042',
    'mcc_cd_3047',
    'mcc_cd_3183',
    'mcc_cd_3217',
    'mcc_cd_3245',
    'mcc_cd_3246',
    'mcc_cd_3301',
    'mcc_cd_3355',
    'mcc_cd_3501',
    'mcc_cd_3502',
    'mcc_cd_3503',
    'mcc_cd_3504',
    'mcc_cd_3509',
    'mcc_cd_3512',
    'mcc_cd_3513',
    'mcc_cd_3530',
    'mcc_cd_3533',
    'mcc_cd_3543',
    'mcc_cd_3553',
    'mcc_cd_3579',
    'mcc_cd_3583',
    'mcc_cd_3598',
    'mcc_cd_3612',
    'mcc_cd_3634',
    'mcc_cd_3640',
    'mcc_cd_3642',
    'mcc_cd_3649',
    'mcc_cd_3665',
    'mcc_cd_3690',
    'mcc_cd_3692',
    'mcc_cd_3710',
    'mcc_cd_3750',
    'mcc_cd_4011',
    'mcc_cd_4119',
    'mcc_cd_4214',
    'mcc_cd_4225',
    'mcc_cd_4411',
    'mcc_cd_4457',
    'mcc_cd_4582',
    'mcc_cd_4722',
    'mcc_cd_4784',
    'mcc_cd_4829',
    'mcc_cd_4899',
    'mcc_cd_5013',
    'mcc_cd_5021',
    'mcc_cd_5044',
    'mcc_cd_5045',
    'mcc_cd_5046',
    'mcc_cd_5047',
    'mcc_cd_5051',
    'mcc_cd_5065',
    'mcc_cd_5072',
    'mcc_cd_5074',
    'mcc_cd_5085',
    'mcc_cd_5094',
    'mcc_cd_5099',
    'mcc_cd_5111',
    'mcc_cd_5122',
    'mcc_cd_5131',
    'mcc_cd_5137',
    'mcc_cd_5139',
    'mcc_cd_5169',
    'mcc_cd_5172',
    'mcc_cd_5192',
    'mcc_cd_5193',
    'mcc_cd_5198',
    'mcc_cd_5199',
    'mcc_cd_5231',
    'mcc_cd_5300',
    'mcc_cd_5309',
    'mcc_cd_5310',
    'mcc_cd_5521',
    'mcc_cd_5531',
    'mcc_cd_5532',
    'mcc_cd_5542',
    'mcc_cd_5551',
    'mcc_cd_5561',
    'mcc_cd_5571',
    'mcc_cd_5599',
    'mcc_cd_5655',
    'mcc_cd_5681',
    'mcc_cd_5697',
    'mcc_cd_5698',
    'mcc_cd_5713',
    'mcc_cd_5718',
    'mcc_cd_5733',
    'mcc_cd_5734',
    'mcc_cd_5735',
    'mcc_cd_5811',
    'mcc_cd_5815',
    'mcc_cd_5817',
    'mcc_cd_5818',
    'mcc_cd_5931',
    'mcc_cd_5932',
    'mcc_cd_5933',
    'mcc_cd_5940',
    'mcc_cd_5946',
    'mcc_cd_5948',
    'mcc_cd_5950',
    'mcc_cd_5960',
    'mcc_cd_5962',
    'mcc_cd_5963',
    'mcc_cd_5965',
    'mcc_cd_5966',
    'mcc_cd_5967',
    'mcc_cd_5968',
    'mcc_cd_5969',
    'mcc_cd_5970',
    'mcc_cd_5971',
    'mcc_cd_5973',
    'mcc_cd_5975',
    'mcc_cd_5976',
    'mcc_cd_5978',
    'mcc_cd_5983',
    'mcc_cd_5994',
    'mcc_cd_5996',
    'mcc_cd_5998',
    'mcc_cd_6050',
    'mcc_cd_6051',
    'mcc_cd_6211',
    'mcc_cd_6513',
    'mcc_cd_6532',
    'mcc_cd_6537',
    'mcc_cd_6540',
    'mcc_cd_7012',
    'mcc_cd_7032',
    'mcc_cd_7033',
    'mcc_cd_7210',
    'mcc_cd_7211',
    'mcc_cd_7216',
    'mcc_cd_7217',
    'mcc_cd_7221',
    'mcc_cd_7251',
    'mcc_cd_7261',
    'mcc_cd_7273',
    'mcc_cd_7277',
    'mcc_cd_7278',
    'mcc_cd_7296',
    'mcc_cd_7297',
    'mcc_cd_7298',
    'mcc_cd_7311',
    'mcc_cd_7333',
    'mcc_cd_7338',
    'mcc_cd_7339',
    'mcc_cd_7349',
    'mcc_cd_7361',
    'mcc_cd_7372',
    'mcc_cd_7375',
    'mcc_cd_7379',
    'mcc_cd_7392',
    'mcc_cd_7393',
    'mcc_cd_7394',
    'mcc_cd_7395',
    'mcc_cd_7399',
    'mcc_cd_742',
    'mcc_cd_7512',
    'mcc_cd_7513',
    'mcc_cd_7519',
    'mcc_cd_7523',
    'mcc_cd_7531',
    'mcc_cd_7534',
    'mcc_cd_7535',
    'mcc_cd_7622',
    'mcc_cd_7623',
    'mcc_cd_7629',
    'mcc_cd_763',
    'mcc_cd_7631',
    'mcc_cd_7641',
    'mcc_cd_7692',
    'mcc_cd_7699',
    'mcc_cd_780',
    'mcc_cd_7829',
    'mcc_cd_7841',
    'mcc_cd_7911',
    'mcc_cd_7929',
    'mcc_cd_7932',
    'mcc_cd_7933',
    'mcc_cd_7941',
    'mcc_cd_7991',
    'mcc_cd_7993',
    'mcc_cd_7994',
    'mcc_cd_7995',
    'mcc_cd_7996',
    'mcc_cd_7998',
    'mcc_cd_7999',
    'mcc_cd_8041',
    'mcc_cd_8042',
    'mcc_cd_8049',
    'mcc_cd_8050',
    'mcc_cd_8071',
    'mcc_cd_8111',
    'mcc_cd_8211',
    'mcc_cd_8220',
    'mcc_cd_8241',
    'mcc_cd_8244',
    'mcc_cd_8249',
    'mcc_cd_8351',
    'mcc_cd_8398',
    'mcc_cd_8641',
    'mcc_cd_8661',
    'mcc_cd_8699',
    'mcc_cd_8734',
    'mcc_cd_8931',
    'mcc_cd_9211',
    'mcc_cd_9223',
    'mcc_cd_9405',
}

MCC_CODES_RARE = {int(e.rsplit('_')[-1]) for e in MCC_CODES_RARE}

df.loc[df['mcc_cd'].isin(MCC_CODES_RARE), 'mcc_cd'] = -1

In [12]:
df.isna().sum(axis=0)

client_id                     0
card_id                       0
tran_time                     0
tran_amt_rur                  0
mcc_cd                        0
merchant_cd                   0
txn_country                   0
txn_city                   5385
tsp_name                   2162
txn_comment_1                 0
txn_comment_2           2638786
tran_time_day_offset          0
mcc_category                  0
txn_comment_1_cat             0
txn_comment_2_cat             0
dtype: int64

In [13]:
df.groupby('client_id')['card_id'].count().sort_values()[::-1]

client_id
 8247514152661289191    3101
-9059132502179716006    2656
 7922361971867610498    2413
-7299536715312723669    2322
 410746725485357352     2318
                        ... 
-293405486348082812        1
 5298349669486843184       1
-6259300834103212263       1
 7627853032776049183       1
 840070581335939466        1
Name: card_id, Length: 11236, dtype: int64

In [14]:
group = df.groupby('client_id').get_group(7922361971867610498)
group.sort_values(by='tran_time', inplace=True)
group.head()

<ipython-input-14-7ee23a22e0ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group.sort_values(by='tran_time', inplace=True)


,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2,tran_time_day_offset,mcc_category,txn_comment_1_cat,txn_comment_2_cat
2940363,7922361971867610498,-5021825852986699015,2018-09-01 00:00:00,72.0,-1,3967370607447365704,IRL,NaN,ITUNES.COM,Payment for goods and services,Purchase payment (web),0,12,5,3
2230234,7922361971867610498,-5021825852986699015,2018-09-01 09:57:00,6071.0,6011,3967370607447365704,RUS,SANKTPETERBU,ATM,Cash withdrawal through an ATM,NaN,0,2,2,-1
2230251,7922361971867610498,-5021825852986699015,2018-09-01 09:58:06,6071.0,6011,3967370607447365704,RUS,SANKTPETERBU,ATM,Cash withdrawal through an ATM,NaN,0,2,2,-1
2728463,7922361971867610498,-5021825852986699015,2018-09-01 10:03:03,284.0,5411,3967370607447365704,RUS,METALLOSTROJ,UNIVERSAM SEMYA,Payment for goods and services,NaN,0,9,5,-1
2728514,7922361971867610498,-5021825852986699015,2018-09-01 10:17:53,607.0,5541,1545852325860792925,RUS,SPETERBURG,AZK,Payment for goods and services,NaN,0,1,5,-1


In [15]:
def create_aggregates_amount_by(group, by):
    kd = group.groupby(by)['tran_amt_rur'].agg(['sum', 'count', 'min', 'max', 'std'])
    kd['std'] = kd['std'].fillna(0)
    kd['mean'] = kd['sum'] / kd['count']
    
    features = {}
    
    for col in kd.columns:
        features_ = {kd.index.name + '_' + str(k) + '_amnt_' + col : v
                     for k, v in kd[col].to_dict().items()}
        features.update(features_)
        
    return features


def create_transaction_features(name, group):
    info = {'client_id': name}
    
    group = group.sort_values(by='tran_time')
    
    info['num_transactions'] = group.shape[0]
    
    tran_time_min = group['tran_time'].min()
    tran_time_max = group['tran_time'].max()
    
    group['days_before'] = (group['tran_time'] - tran_time_max).dt.days
    
    info['num_weekofyear'] = group['tran_time'].dt.isocalendar().week.nunique()
    info['weekofyear_last'] = group['tran_time'].dt.isocalendar().week.iloc[-1]
    
    info['num_days_tran_diff'] = (tran_time_max - tran_time_min).days
    info['num_days_tran_uniq'] = group['tran_time_day_offset'].nunique()
    info['num_days_tran_median'] = group['tran_time_day_offset'].median()
    info['num_days_tran_max'] = group['tran_time_day_offset'].max()
    info['num_days_tran_std'] = group['tran_time_day_offset'].std()
    
    hour_diff = group['tran_time'].diff().dt.total_seconds() / 3600
    
    info['hour_mean'] = group['tran_time'].dt.hour.mean()
    info['hour_std'] = group['tran_time'].dt.hour.std()
    info['hour_q05'], info['hour_q50'], info['hour_q95'] = \
        group['tran_time'].dt.hour.quantile([0.05, 0.5, 0.95])

    info['hour_diff_q50'] = hour_diff.median()
    info['hour_diff_max'] = hour_diff.max()
    info['hour_diff_std'] = hour_diff.std()
    
    info['weekend_amnt_mean'] = group.loc[group['tran_time'].dt.weekday >= 5, 'tran_amt_rur'].mean()
    
    amnt = group['tran_amt_rur']
    
    info['amnt_sum'] = amnt.sum()
    info['amnt_mean'] = amnt.mean()
    info['amnt_max'] = amnt.max()
    info['amnt_std'] = amnt.std()
    
    info['amnt_q10'], info['amnt_q50'], info['amnt_q95'] = \
        amnt.quantile([0.1, 0.5, 0.95])
    
    info['amnt_mean_last_month'] = amnt[group['days_before'] < 30].mean()
    if np.isnan(info['amnt_mean_last_month']):
        info['amnt_mean_last_month'] = 0
    
    info['div(amnt_mean_last_month__amnt_mean)'] = \
        info['amnt_mean_last_month'] / (info['amnt_mean'] + 1)
    
    amnt_diff = group['tran_amt_rur'].diff()
    
    info['amnt_diff_std'] = amnt_diff.std()
    info['amnt_diff_q10'], info['amnt_diff_q50'], info['amnt_diff_q95'] = \
        amnt_diff.quantile([0.1, 0.5, 0.95])

    info['num_cards'] = group['card_id'].nunique()
    
    info.update(create_aggregates_amount_by(group, by='mcc_cd'))
    info.update(create_aggregates_amount_by(group, by='mcc_category'))
    info.update(create_aggregates_amount_by(group, by='txn_comment_1_cat'))
    info.update(create_aggregates_amount_by(group, by='txn_comment_2_cat'))
    
    return info

In [16]:
create_transaction_features(0, group)

{'client_id': 0,
 'num_transactions': 2413,
 'num_weekofyear': 52,
 'weekofyear_last': 35,
 'num_days_tran_diff': 364,
 'num_days_tran_uniq': 362,
 'num_days_tran_median': 186.0,
 'num_days_tran_max': 364,
 'num_days_tran_std': 104.95985063276596,
 'hour_mean': 13.891007045171985,
 'hour_std': 5.133230552893014,
 'hour_q05': 0.0,
 'hour_q50': 14.0,
 'hour_q95': 20.0,
 'hour_diff_q50': 0.8604166666666666,
 'hour_diff_max': 41.33972222222222,
 'hour_diff_std': 5.717851412023088,
 'weekend_amnt_mean': 728.9023668639053,
 'amnt_sum': 1812590.0,
 'amnt_mean': 751.1769581433899,
 'amnt_max': 82803.0,
 'amnt_std': 2421.098278412274,
 'amnt_q10': 73.0,
 'amnt_q50': 279.0,
 'amnt_q95': 2601.6000000000013,
 'amnt_mean_last_month': 751.1769581433899,
 'div(amnt_mean_last_month__amnt_mean)': 0.9986705256134563,
 'amnt_diff_std': 3338.0848147021734,
 'amnt_diff_q10': -977.4999999999999,
 'amnt_diff_q50': -1.0,
 'amnt_diff_q95': 1922.4499999999998,
 'num_cards': 3,
 'num_merchant_cd': 121,
 'mcc_cd_

In [17]:
def make_aggregates(df, verbose=False):
    df_grouped = df.groupby('client_id')
    if verbose:
        df_grouped = tqdm(df_grouped)
    
    with Pool(processes=NUM_THREADS) as pool:
        df_features = pool.starmap(create_transaction_features, df_grouped, chunksize=100)
        df_features = pd.DataFrame(df_features)
        
    return df_features

In [18]:
from multiprocessing import Pool, cpu_count

# NUM_THREADS = cpu_count()
NUM_THREADS = 2

df_features = make_aggregates(df, verbose=True)

  0%|          | 0/11236 [00:00<?, ?it/s]

/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/v.bugaevskii/.conda/envs/idao2021/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/v.bugaevskii/.conda/envs/idao

In [19]:
df_features.head()

,client_id,num_transactions,num_weekofyear,weekofyear_last,num_days_tran_diff,num_days_tran_uniq,num_days_tran_median,num_days_tran_max,num_days_tran_std,hour_mean,...,mcc_cd_5714_amnt_min,mcc_cd_5714_amnt_max,mcc_cd_5714_amnt_std,mcc_cd_5714_amnt_mean,txn_comment_2_cat_1_amnt_sum,txn_comment_2_cat_1_amnt_count,txn_comment_2_cat_1_amnt_min,txn_comment_2_cat_1_amnt_max,txn_comment_2_cat_1_amnt_std,txn_comment_2_cat_1_amnt_mean
0,-9220236243053692422,101,15,35,129,33,313.0,364,37.008209,14.990099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9220233431709087652,447,50,34,357,182,106.0,358,109.263280,10.939597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-9219699286371310531,11,10,30,275,10,209.0,330,90.733476,1.545455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-9218923580775423458,87,11,35,98,43,302.0,361,24.232573,5.701149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-9218871523310554579,49,28,33,327,38,241.0,351,101.262973,14.673469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_features.isna().sum(axis=0)

client_id                             0
num_transactions                      0
num_weekofyear                        0
weekofyear_last                       0
num_days_tran_diff                    0
                                  ...  
txn_comment_2_cat_1_amnt_count    11233
txn_comment_2_cat_1_amnt_min      11233
txn_comment_2_cat_1_amnt_max      11233
txn_comment_2_cat_1_amnt_std      11233
txn_comment_2_cat_1_amnt_mean     11233
Length: 759, dtype: int64

In [21]:
df_features.columns[df_features.isna().sum(axis=0) > 0]

Index(['num_days_tran_std', 'hour_std', 'hour_diff_q50', 'hour_diff_max',
       'hour_diff_std', 'weekend_amnt_mean', 'amnt_std', 'amnt_diff_std',
       'amnt_diff_q10', 'amnt_diff_q50',
       ...
       'mcc_cd_5714_amnt_min', 'mcc_cd_5714_amnt_max', 'mcc_cd_5714_amnt_std',
       'mcc_cd_5714_amnt_mean', 'txn_comment_2_cat_1_amnt_sum',
       'txn_comment_2_cat_1_amnt_count', 'txn_comment_2_cat_1_amnt_min',
       'txn_comment_2_cat_1_amnt_max', 'txn_comment_2_cat_1_amnt_std',
       'txn_comment_2_cat_1_amnt_mean'],
      dtype='object', length=737)

In [22]:
counts = df_features.isna().sum(axis=0) / df_features.shape[0]
columns = df_features.columns[counts < 0.95]
columns

Index(['client_id', 'num_transactions', 'num_weekofyear', 'weekofyear_last',
       'num_days_tran_diff', 'num_days_tran_uniq', 'num_days_tran_median',
       'num_days_tran_max', 'num_days_tran_std', 'hour_mean',
       ...
       'mcc_cd_5261_amnt_min', 'mcc_cd_5261_amnt_max', 'mcc_cd_5261_amnt_std',
       'mcc_cd_5261_amnt_mean', 'mcc_cd_5714_amnt_sum',
       'mcc_cd_5714_amnt_count', 'mcc_cd_5714_amnt_min',
       'mcc_cd_5714_amnt_max', 'mcc_cd_5714_amnt_std',
       'mcc_cd_5714_amnt_mean'],
      dtype='object', length=747)

In [23]:
set(df_features.columns[counts >= 0.95].str.rsplit('_', 2).str[0])

{'txn_comment_1_cat_1', 'txn_comment_2_cat_1'}

In [24]:
counts[counts.index.str.startswith('txn_comment_')]

txn_comment_1_cat_2_amnt_sum      0.144001
txn_comment_1_cat_3_amnt_sum      0.398896
txn_comment_1_cat_5_amnt_sum      0.069776
txn_comment_1_cat_2_amnt_count    0.144001
txn_comment_1_cat_3_amnt_count    0.398896
                                    ...   
txn_comment_2_cat_1_amnt_count    0.999733
txn_comment_2_cat_1_amnt_min      0.999733
txn_comment_2_cat_1_amnt_max      0.999733
txn_comment_2_cat_1_amnt_std      0.999733
txn_comment_2_cat_1_amnt_mean     0.999733
Length: 78, dtype: float64

In [25]:
from functools import reduce
from operator import or_

mask = reduce(or_, [
    df_features.columns.str.startswith('txn_comment_1_cat_1'),
    df_features.columns.str.startswith('txn_comment_2_cat_1'),
])
columns = df_features.columns[mask]

df_features.drop(columns=columns, inplace=True)

In [26]:
df_features.to_csv('trxn_features_2.csv', index=False)

In [27]:
df_features.shape

(11236, 747)